In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [62]:
import io

import numpy as np
import pymongo

from bokeh.io import output_notebook, show
from bokeh.palettes import all_palettes
from bokeh.layouts import gridplot

from IPython import display

#import climatools.lblnew.mongo as mongo
from climatools.cliradlw.dataset import load_output_file
import climatools.html.html as climahtml
from climatools.plot.plot import *

In [35]:
output_notebook()

Loading BokehJS ...

In [2]:
%run param.py

In [3]:
PARAM

{'atmpro': 'mls',
 'band': '3b',
 'commitnumber': 'a22ab94',
 'conc': 0.0008,
 'dv': 0.001,
 'klin': 0,
 'molecule': 'co2',
 'ng_adju': [0, 0],
 'ng_refs': [5, 2],
 'nv': 1000,
 'option_compute_btable': 0,
 'option_compute_ktable': 0,
 'option_wgt_flux': 1,
 'option_wgt_k': 1,
 'ref_pts': [(1, 250), (10, 250)],
 'tsfc': 294,
 'vmax': 720,
 'vmin': 620,
 'w_diffuse': [(1.66, 1.66, 1.66, 1.66, 1.66), (1.66, 1.66)],
 'wgt': [(0, 0.6, 0.5, 0.7, 0.8), (0.8, 0.7)]}

In [6]:
client = pymongo.MongoClient('localhost', 27017, connect=False)

In [7]:
db = client.cliradlw

In [19]:
db.test_collection.count()

6

In [20]:

def make_query(param=None):
    '''
    Returns the MongoDB query for a lblnew-bestfit 
    run's document.
    
    Parameters
    ----------
    param: dict
    lblnew-bestfit input parameters.
    '''
    return {'param.' + name: value for name, value in param.items()}
    

In [21]:
doc = db.test_collection.find_one(make_query(param=PARAM))

In [22]:
doc.keys()

dict_keys(['_id', 'param', 'output_wfluxg', 'output_wcoolrg', 'output_fluxg', 'output_coolrg', 'output_flux', 'output_coolr'])

In [72]:
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

In [56]:
def print_input_param():
    for name, value in sorted(PARAM.items()):
        print(f'{name} = {value}')

In [95]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
        
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band') 
            
    return ds['coolrg']



def pltdata_cool(g=None):
    colors = all_palettes['Set1'][4]
    
    if g == None:
        ds_cool_crd = DS_COOL_CRD.sum('g')
        ds_cool_wgt = DS_COOL_WGT.sum('g')
    else:
        ds_cool_crd = DS_COOL_CRD.sel(g=g)
        ds_cool_wgt = DS_COOL_WGT.sel(g=g)
    
    data = [
        {'label': 'CRD',
         'srs': fmt_cool(ds_cool_crd),
         'line_dash': 'solid', 'line_width':1.5,
         'marker': 'circle', 'mark_size': 5,
         'color': colors[2], 'alpha': 1},
        {'label': 'WGT igg=1',
         'srs': fmt_cool(ds_cool_wgt.sel(igg=1)),
         'line_dash': 'solid', 'line_width': 3,
         'marker': 'square', 'marker_size': 5,
         'color': colors[3], 'alpha': .6},
        {'label': 'WGT igg=10',
         'srs': fmt_cool(ds_cool_wgt.sel(igg=10)),
         'line_dash': 'solid', 'line_width': 2,
         'marker': 'triangle', 'marker_size': 2,
         'color': colors[1], 'alpha': .3}]
    return data



def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata, 
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200)) 
    plog.plot_width, plog.plot_height = 400, 500
    
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
    

def script_plt_cool():
    ng = sum(PARAM['ng_refs'])
    for g in np.arange(1, ng + 1):
        data = pltdata_cool(g=g)
        p = plt_cool(pltdata=data)
        show(p)
        display.display(
            display.Markdown(f'*FIGURE.* Cooling rates. g={g}'))
    
    data = pltdata_cool()
    p = plt_cool(pltdata=data)
    show(p)
    display.display(
        display.Markdown(f'*FIGURE.* Cooling rates. Summed over g.'))
    


In [111]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.squeeze('igg').drop('igg')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=1',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=[1]), ilevels=ilevels)},
           {'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=[10]), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data



def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        

        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)
    
    
tbdata_flux()

{'benchmark': {'df':                        flug       fldg     fnetg
  g pressure level                                
  1 0.000    1      -0.015126   0.000000 -0.015126
    180.875  41     -0.018037   0.017026 -0.001011
    1013.000 76     -0.056422   0.056155 -0.000267
  2 0.000    1      -0.065087   0.000000 -0.065087
    180.875  41     -0.059987   0.056631 -0.003356
    1013.000 76     -0.187351   0.186463 -0.000887
  3 0.000    1      -0.530092   0.000000 -0.530092
    180.875  41     -0.282368   0.265968 -0.016399
    1013.000 76     -0.879953   0.875766 -0.004188
  4 0.000    1      -1.827432   0.000000 -1.827432
    180.875  41     -0.939556   0.879312 -0.060243
    1013.000 76     -2.906766   2.892698 -0.014068
  5 0.000    1      -4.790672   0.000000 -4.790672
    180.875  41     -3.474725   3.202560 -0.272166
    1013.000 76    -10.707804  10.655217 -0.052586
  6 0.000    1      -7.221282   0.000000 -7.221282
    180.875  41     -6.919034   5.988656 -0.930379
    1013.000

In [92]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

In [99]:
def script():
    title = '### lblnew-bestfit'
    s_input = 'Input Parameters'
    s_plt_cool = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    
    # title
    show_markdown(title)
    
    # TOC
    show_markdown('### Table of Contents')
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cool))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cool))
    #script_plt_cool()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    

In [112]:
script()

### lblnew-bestfit

### Table of Contents

# Input Parameters

atmpro = mls
band = 3b
commitnumber = a22ab94
conc = 0.0008
dv = 0.001
klin = 0
molecule = co2
ng_adju = [0, 0]
ng_refs = [5, 2]
nv = 1000
option_compute_btable = 0
option_compute_ktable = 0
option_wgt_flux = 1
option_wgt_k = 1
ref_pts = [(1, 250), (10, 250)]
tsfc = 294
vmax = 720
vmin = 620
w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66), (1.66, 1.66)]
wgt = [(0, 0.6, 0.5, 0.7, 0.8), (0.8, 0.7)]


# Cooling Rate Profiles

...

Table. Fluxes. WGT igg=10


flug      fldg         fnetg
g pressure level                                  
1 0.000    1     -0.000052  0.000000 -5.188600e-05
  180.875  41     0.000006 -0.000005  1.412000e-07
  1013.000 76    -0.000041  0.000041 -6.100100e-07
2 0.000    1      0.007109  0.000000  7.108983e-03
  180.875  41     0.000069 -0.000071 -1.996000e-06
  1013.000 76    -0.000281  0.000278 -3.155270e-06
3 0.000    1      0.012062  0.000000  1.206232e-02
  180.875  41     0.000927 -0.000302  6.246010e-04
  1013.000 76    -0.001366  0.001373  6.290400e-06
4 0.000    1     -0.068513  0.000000 -6.851340e-02
  180.875  41     0.009914 -0.001776  8.137534e-03
  1013.000 76    -0.004365  0.004621  2.563180e-04
5 0.000    1     -0.102440  0.000000 -1.024403e-01
  180.875  41     0.050877  0.029384  8.026144e-02
  1013.000 76    -0.013822  0.015540  1.716981e-03
6 0.000    1      0.053222  0.000000  5.322230e-02
  180.875  41     0.211378  0.268488  4.798657e-01
  1013.000 76    -0.005587  0.010755  5.167116e-03
7 0.000    1      0.117531  0.000000  1.175309e-01
  180.875  41     0.071988 -0.068657  3.330700e-03
  1013.000 76     0.025464 -0.001148  2.431597e-02

Table. Fluxes. (WGT igg=1) - (CRD)


flug      fldg         fnetg
g pressure level                                  
1 0.000    1     -0.000029  0.000000 -2.930900e-05
  180.875  41     0.000006 -0.000005  1.412000e-07
  1013.000 76    -0.000041  0.000041 -6.100100e-07
2 0.000    1      0.007766  0.000000  7.766150e-03
  180.875  41     0.000069 -0.000071 -1.996000e-06
  1013.000 76    -0.000281  0.000278 -3.155270e-06
3 0.000    1      0.011400  0.000000  1.139977e-02
  180.875  41     0.000927 -0.000302  6.246010e-04
  1013.000 76    -0.001366  0.001373  6.290400e-06
4 0.000    1     -0.084368  0.000000 -8.436770e-02
  180.875  41     0.009914 -0.001776  8.137534e-03
  1013.000 76    -0.004365  0.004621  2.563180e-04
5 0.000    1     -0.092756  0.000000 -9.275590e-02
  180.875  41     0.050877  0.029384  8.026144e-02
  1013.000 76    -0.013822  0.015540  1.716981e-03
6 0.000    1      0.082646  0.000000  8.264600e-02
  180.875  41     0.234450  0.273266  5.077159e-01
  1013.000 76    -0.005587  0.010753  5.165328e-03
7 0.000    1      0.182656  0.000000  1.826562e-01
  180.875  41     0.114622 -0.085163  2.945960e-02
  1013.000 76     0.025464  0.006154  3.161797e-02

Table. Fluxes. (WGT igg=10) - (CRD)
